In [7]:
#install.packages("lme4")
#install.packages("tidyverse")

In [92]:
library(lme4)
library(purrr)
library(tibble)

In [140]:
dat <- read.csv('../data/mehrabi-eval/data/analysed_data_final_bark.csv')
dat$imitator <- as.factor(dat$imitator)
dat$listener <- as.factor(dat$listener)
dat$imitated_sound <- as.factor(dat$imitated_sound)
dat$rated_sound <- as.factor(dat$rated_sound)
dat$rating <- as.double(dat$rating) / 100
head(dat)

,X,listener,imitator,imitated_sound,rated_sound,rating,distance_mfcc,distance_square.1,distance_square.2,distance_square.3,distance_tall.1,distance_tall.2,distance_tall.3,distance_tall.4,distance_wide.1,distance_wide.2,distance_wide.3,distance_wide.4,distance_mfcc_short
,<int>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,12,6,20,19,0.00,0.06332231,0.2087883,0.2192959,0.3276160,0.2418618,0.2502268,0.2480478,0.2704551,0.2052346,0.2473502,0.2273675,0.2164745,0.2937442
2,1,12,6,20,18,0.00,0.09943612,0.1892278,0.2037417,0.2799401,0.2335096,0.2516639,0.2278006,0.2488989,0.1929450,0.2389742,0.2006590,0.1944595,0.2829110
3,2,12,6,20,21,0.41,0.14185922,0.5562489,0.6218457,0.6495880,0.5950477,0.5747121,0.5116193,0.5324236,0.5987752,0.6204424,0.6126037,0.5143839,0.1958800
4,3,12,6,20,20,0.40,0.41093785,0.3207782,0.3261316,0.3998655,0.3684330,0.3790807,0.3512518,0.3725511,0.3443145,0.3714650,0.3745479,0.3561645,0.4982607
5,4,12,6,20,23,0.00,0.36282176,0.5641272,0.6378974,0.7255492,0.6183479,0.6544065,0.5800459,0.6344287,0.6349404,0.6600428,0.6554899,0.5932593,0.2622058
6,5,12,6,20,22,0.55,0.37178692,0.5188829,0.5670232,0.6312122,0.5561374,0.5567452,0.5058479,0.5446006,0.5628097,0.6201336,0.6212298,0.5188746,0.2710403


In [141]:
cols <- colnames(dat) %>% keep(~ startsWith(., "distance"))
df <- data.frame(name=cols, AIC=double(13), Accuracy=double(13))
str(df)

'data.frame':	13 obs. of  3 variables:
 $ name    : Factor w/ 13 levels "distance_mfcc",..: 1 3 4 5 6 7 8 9 10 11 ...
 $ AIC     : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Accuracy: num  0 0 0 0 0 0 0 0 0 0 ...


In [148]:
for (col in cols) {
    formula <- substitute(rating ~ 0 + imitated_sound + (imitated_sound:dist) + (1 | listener), list(dist = as.name(col)))
    #formula <- substitute(rating ~ imitated_sound*dist + (1 | listener), list(dist = as.name(col)))
    model <- lmer(formula, dat, REML=FALSE)
    aic <- extractAIC(model)
    df[df$name == col, 2] <- aic[2]
    intervals <- confint(model, method="Wald")
    intervals <- intervals[33:62, "97.5 %"]
    neg_intervals <- intervals %>% keep(~ .x < 0.0)
    df[df$name == col, 3] <- length(neg_intervals) / length(intervals)
}

In [149]:
df

name,AIC,Accuracy
<fct>,<dbl>,<dbl>
distance_mfcc,1544.808274,0.3000000
distance_square.1,63.357831,0.9000000
distance_square.2,14.535779,0.9000000
distance_square.3,-32.434946,0.9000000
distance_tall.1,-110.309544,0.9000000
distance_tall.2,-6.393861,0.8666667
distance_tall.3,118.805262,0.8333333
distance_tall.4,23.541077,0.8333333
distance_wide.1,-50.146008,0.9000000
